**Notes**:

Success except for tflite

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os

In [5]:
sys.version

'3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) \n[GCC 7.2.0]'

In [6]:
from fastai.vision import *

In [7]:
torch.__version__

'1.0.1.post2'

In [8]:
import torchvision

In [9]:
torchvision.__version__

'0.2.1'

In [10]:
import tensorflow as tf

In [11]:
tf.__version__

'1.13.1'

In [12]:
import onnx

In [13]:
onnx.__version__

'1.4.1'

In [14]:
[m for m in dir(models) if m[:2] != '__']

['BasicBlock',
 'Darknet',
 'DynamicUnet',
 'ResLayer',
 'ResNet',
 'SqueezeNet',
 'UnetBlock',
 'WideResNet',
 'XResNet',
 'alexnet',
 'darknet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'squeezenet1_0',
 'squeezenet1_1',
 'unet',
 'vgg16_bn',
 'vgg19_bn',
 'wrn',
 'wrn_22',
 'xception',
 'xresnet',
 'xresnet101',
 'xresnet152',
 'xresnet18',
 'xresnet34',
 'xresnet50']

In [88]:
from torchvision.models import vgg11
models.vgg11 = vgg11

### Paths

In [ ]:
MODELS = Path('./models/'); MODELS.ls()
arch_name = 'vgg11'

def inputs_same(x1,x2): return np.all(np.isclose(x1, x2, atol=1e-5))

### Load model

In [91]:
f = getattr(models, arch_name)
model = f(True); model.eval();

Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /home/ubuntu/.torch/models/vgg11-bbd30ac9.pth
531456000it [00:33, 15797853.37it/s]


### Add permute for tflite

In [93]:
class Model(nn.Module):
    def __init__(self, model=model):
        super().__init__()
        self.model = model
    
    def forward(self, x):    
        x = x.permute(0,3,1,2).contiguous()
        return self.model(x)

In [94]:
model = Model(model)

In [96]:
model

Model(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace)
      (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [126]:
from fastai.layers import *

In [156]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0)
        
    def forward(self, x):
        x = x.permute(0,3,1,2).contiguous()
        return self.conv1(x)

In [157]:
model = SimpleModel()

### Get input ready

In [158]:
img = open_image("cat224x224.jpg")

In [159]:
normalized_img = normalize(img.data, tensor(imagenet_stats[0]), tensor(imagenet_stats[1]))

In [160]:
# batch x channel x height x width
torch_input = normalized_img[None, ...]
numpy_input = to_np(torch_input)

In [161]:
torch_input.shape

torch.Size([1, 3, 224, 224])

In [162]:
torch_input = torch_input.permute(0,2,3,1)
numpy_input = to_np(torch_input)

In [163]:
torch_input.shape, numpy_input.shape

(torch.Size([1, 224, 224, 3]), (1, 224, 224, 3))

### pytorch output

In [164]:
if torch.cuda.is_available():
    with torch.cuda.device(0):
        torch_output = to_np(model(torch_input))
else: torch_output = to_np(model(torch_input))

In [165]:
np.argmax(torch_output[0])

2063640

In [166]:
torch_output[0]

array([[[ 5.928250e-01,  6.495560e-01,  6.586254e-01,  6.328704e-01, ...,  1.145797e-01, -1.446192e-02,
         -9.115485e-02, -1.700499e-01],
        [ 6.237740e-01,  6.704966e-01,  6.704947e-01,  6.700016e-01, ...,  1.083156e-01, -3.941596e-02,
         -9.333248e-02, -1.599272e-01],
        [ 6.543528e-01,  7.007562e-01,  6.936352e-01,  7.207102e-01, ...,  1.495145e-01, -5.462829e-04,
         -5.724194e-02, -1.157975e-01],
        [ 6.586937e-01,  6.734817e-01,  6.918388e-01,  7.479682e-01, ...,  1.761432e-01,  5.087249e-02,
         -1.233188e-02, -3.808299e-02],
        ...,
        [-5.286742e-01, -3.994761e-01, -4.185633e-01, -5.430715e-01, ..., -8.238600e-01, -8.971226e-01,
         -1.071087e+00, -1.090646e+00],
        [-6.421031e-01, -5.654278e-01, -4.625924e-01, -5.476409e-01, ..., -8.586741e-01, -7.533782e-01,
         -8.619232e-01, -9.086895e-01],
        [-6.917681e-01, -6.695323e-01, -5.033025e-01, -4.304609e-01, ..., -8.144889e-01, -6.926728e-01,
         -7.351253e

### pytorch to onnx

In [167]:
onnx_output = torch.onnx._export(model, torch_input, MODELS/f'{arch_name}.onnx', verbose=False)

In [168]:
# Load the ONNX model
model = onnx.load(MODELS/f"{arch_name}.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

onnx_output = to_np(onnx_output)[0]

graph torch-jit-export (
  %0[FLOAT, 1x224x224x3]
) initializers (
  %1[FLOAT, 64x3x3x3]
  %2[FLOAT, 64]
) {
  %3 = Transpose[perm = [0, 3, 1, 2]](%0)
  %4 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [1, 1]](%3, %1, %2)
  return %4
}


In [169]:
torch_output[0]

array([[[ 5.928250e-01,  6.495560e-01,  6.586254e-01,  6.328704e-01, ...,  1.145797e-01, -1.446192e-02,
         -9.115485e-02, -1.700499e-01],
        [ 6.237740e-01,  6.704966e-01,  6.704947e-01,  6.700016e-01, ...,  1.083156e-01, -3.941596e-02,
         -9.333248e-02, -1.599272e-01],
        [ 6.543528e-01,  7.007562e-01,  6.936352e-01,  7.207102e-01, ...,  1.495145e-01, -5.462829e-04,
         -5.724194e-02, -1.157975e-01],
        [ 6.586937e-01,  6.734817e-01,  6.918388e-01,  7.479682e-01, ...,  1.761432e-01,  5.087249e-02,
         -1.233188e-02, -3.808299e-02],
        ...,
        [-5.286742e-01, -3.994761e-01, -4.185633e-01, -5.430715e-01, ..., -8.238600e-01, -8.971226e-01,
         -1.071087e+00, -1.090646e+00],
        [-6.421031e-01, -5.654278e-01, -4.625924e-01, -5.476409e-01, ..., -8.586741e-01, -7.533782e-01,
         -8.619232e-01, -9.086895e-01],
        [-6.917681e-01, -6.695323e-01, -5.033025e-01, -4.304609e-01, ..., -8.144889e-01, -6.926728e-01,
         -7.351253e

In [170]:
onnx_output

array([[[ 5.928250e-01,  6.495560e-01,  6.586254e-01,  6.328704e-01, ...,  1.145797e-01, -1.446192e-02,
         -9.115485e-02, -1.700499e-01],
        [ 6.237740e-01,  6.704966e-01,  6.704947e-01,  6.700016e-01, ...,  1.083156e-01, -3.941596e-02,
         -9.333248e-02, -1.599272e-01],
        [ 6.543528e-01,  7.007562e-01,  6.936352e-01,  7.207102e-01, ...,  1.495145e-01, -5.462829e-04,
         -5.724194e-02, -1.157975e-01],
        [ 6.586937e-01,  6.734817e-01,  6.918388e-01,  7.479682e-01, ...,  1.761432e-01,  5.087249e-02,
         -1.233188e-02, -3.808299e-02],
        ...,
        [-5.286742e-01, -3.994761e-01, -4.185633e-01, -5.430715e-01, ..., -8.238600e-01, -8.971226e-01,
         -1.071087e+00, -1.090646e+00],
        [-6.421031e-01, -5.654278e-01, -4.625924e-01, -5.476409e-01, ..., -8.586741e-01, -7.533782e-01,
         -8.619232e-01, -9.086895e-01],
        [-6.917681e-01, -6.695323e-01, -5.033025e-01, -4.304609e-01, ..., -8.144889e-01, -6.926728e-01,
         -7.351253e

In [171]:
if not inputs_same(torch_output[0], onnx_output):
    raise Exception("PyTorch and ONNX outputs are not same")

### onnx to coreml

In [172]:
from onnx_coreml import convert

In [193]:
import onnx_coreml

In [173]:
# convert to coreml
model = onnx.load(MODELS/f"{arch_name}.onnx")
mlmodel = convert(model)

1/2: Converting Node Type Transpose
2/2: Converting Node Type Conv
Translation to CoreML spec completed. Now compiling the CoreML model.
Model Compilation done.


In [174]:
mlmodel.save(MODELS/f'{arch_name}.mlmodel')

In [175]:
# Exception: Model prediction is only supported on macOS version 10.13 or later.
# torch with coreml - only on mac OS
# mlmodel.predict({"0":numpy_input})

### onnx to tf 

You should get an warning message says

UserWarning: Using the pooling op in compatibility mode.This means your graph cannot be serialized.
Please configure your pooling operation to only use paddings that correspond to Tensorflow SAME or VALID padding.
One layer in your network can not use native tensorflow op. We have to use compatible pool.
I checked and it seems following layer:

In [176]:
from onnx_tf.backend import prepare

In [177]:
model = onnx.load(MODELS/f"{arch_name}.onnx")
tf_rep = prepare(model)
tf_output = tf_rep.run(numpy_input)

/home/ubuntu/onnx-tensorflow/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/home/ubuntu/onnx-tensorflow/onnx_tf/common/handler_helper.py:73: UserWarning: Unknown op MeanVarianceNormalization in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))


In [178]:
tf_rep.graph.get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation '0' type=Placeholder>,
 <tf.Operation 'transpose/perm' type=Const>,
 <tf.Operation 'transpose' type=Transpose>,
 <tf.Operation 'transpose_1/perm' type=Const>,
 <tf.Operation 'transpose_1' type=Transpose>,
 <tf.Operation 'Const_2' type=Const>,
 <tf.Operation 'Pad' type=Pad>,
 <tf.Operation 'Const_3' type=Const>,
 <tf.Operation 'split/split_dim' type=Const>,
 <tf.Operation 'split' type=Split>,
 <tf.Operation 'transpose_2/perm' type=Const>,
 <tf.Operation 'transpose_2' type=Transpose>,
 <tf.Operation 'Const_4' type=Const>,
 <tf.Operation 'split_1/split_dim' type=Const>,
 <tf.Operation 'split_1' type=Split>,
 <tf.Operation 'convolution/dilation_rate' type=Const>,
 <tf.Operation 'convolution' type=Conv2D>,
 <tf.Operation 'concat/concat_dim' type=Const>,
 <tf.Operation 'concat' type=Identity>,
 <tf.Operation 'Add' type=Add>,
 <tf.Operation 'transpose_3/perm' type=Const>,
 <tf.Operation 'transpose_3' type

In [179]:
tf_output[0][0]

array([[[ 5.928249e-01,  6.495559e-01,  6.586254e-01,  6.328702e-01, ...,  1.145797e-01, -1.446191e-02,
         -9.115484e-02, -1.700499e-01],
        [ 6.237740e-01,  6.704966e-01,  6.704946e-01,  6.700016e-01, ...,  1.083155e-01, -3.941594e-02,
         -9.333252e-02, -1.599272e-01],
        [ 6.543528e-01,  7.007561e-01,  6.936354e-01,  7.207103e-01, ...,  1.495145e-01, -5.463138e-04,
         -5.724193e-02, -1.157975e-01],
        [ 6.586937e-01,  6.734816e-01,  6.918389e-01,  7.479682e-01, ...,  1.761432e-01,  5.087247e-02,
         -1.233190e-02, -3.808297e-02],
        ...,
        [-5.286742e-01, -3.994760e-01, -4.185633e-01, -5.430715e-01, ..., -8.238600e-01, -8.971226e-01,
         -1.071087e+00, -1.090645e+00],
        [-6.421031e-01, -5.654277e-01, -4.625923e-01, -5.476409e-01, ..., -8.586740e-01, -7.533783e-01,
         -8.619232e-01, -9.086896e-01],
        [-6.917681e-01, -6.695324e-01, -5.033025e-01, -4.304608e-01, ..., -8.144889e-01, -6.926728e-01,
         -7.351254e

In [180]:
torch_output[0]

array([[[ 5.928250e-01,  6.495560e-01,  6.586254e-01,  6.328704e-01, ...,  1.145797e-01, -1.446192e-02,
         -9.115485e-02, -1.700499e-01],
        [ 6.237740e-01,  6.704966e-01,  6.704947e-01,  6.700016e-01, ...,  1.083156e-01, -3.941596e-02,
         -9.333248e-02, -1.599272e-01],
        [ 6.543528e-01,  7.007562e-01,  6.936352e-01,  7.207102e-01, ...,  1.495145e-01, -5.462829e-04,
         -5.724194e-02, -1.157975e-01],
        [ 6.586937e-01,  6.734817e-01,  6.918388e-01,  7.479682e-01, ...,  1.761432e-01,  5.087249e-02,
         -1.233188e-02, -3.808299e-02],
        ...,
        [-5.286742e-01, -3.994761e-01, -4.185633e-01, -5.430715e-01, ..., -8.238600e-01, -8.971226e-01,
         -1.071087e+00, -1.090646e+00],
        [-6.421031e-01, -5.654278e-01, -4.625924e-01, -5.476409e-01, ..., -8.586741e-01, -7.533782e-01,
         -8.619232e-01, -9.086895e-01],
        [-6.917681e-01, -6.695323e-01, -5.033025e-01, -4.304609e-01, ..., -8.144889e-01, -6.926728e-01,
         -7.351253e

In [181]:
# torch with TF
if not inputs_same(torch_output[0], tf_output[0][0]):
    raise Exception("PyTorch and TF outputs are not same")

In [182]:
tf_rep.export_graph(MODELS/f"{arch_name}.pb")

In [183]:
tf_rep.inputs[0], tf_rep.outputs[0]

('0', '4')

In [184]:
tf_rep.tensor_dict[tf_rep.inputs[0]], tf_rep.tensor_dict[ tf_rep.outputs[0]]

(<tf.Tensor '0:0' shape=(1, 224, 224, 3) dtype=float32>,
 <tf.Tensor 'transpose_3:0' shape=(1, 64, 222, 222) dtype=float32>)

### ONNX caffe2 backend

In [185]:
import caffe2.python.onnx.backend as onnx_caffe2_backend

# content image shape 224, 224, 3
model = onnx.load(MODELS/f'{arch_name}.onnx')
prepared_backend = onnx_caffe2_backend.prepare(model, device='CUDA' if torch.cuda.is_available() else 'CPU')

In [186]:
inp = {model.graph.input[0].name: numpy_input.astype(np.float32)}
caffe2_output = prepared_backend.run(inp)[0]

In [187]:
# torch with TF
if not inputs_same(torch_output[0], caffe2_output[0]):
    raise Exception("PyTorch and Caffe2 outputs are not same")

### ONNX to caffe2 pb

In [188]:
# extract the workspace and the model proto from the internal representation
c2_workspace = prepared_backend.workspace
c2_model = prepared_backend.predict_net

# Now import the caffe2 mobile exporter
from caffe2.python.predictor import mobile_exporter

# call the Export to get the predict_net, init_net. These nets are needed for running things on mobile
init_net, predict_net = mobile_exporter.Export(c2_workspace, c2_model, c2_model.external_input)

# Let's also save the init_net and predict_net to a file that we will later use for running them on mobile
with open(MODELS/f'{arch_name}_init_net.pb', "wb") as fopen:
    fopen.write(init_net.SerializeToString())
with open(MODELS/f'{arch_name}_net.pb', "wb") as fopen:
    fopen.write(predict_net.SerializeToString())

In [189]:
# Some standard imports
from caffe2.proto import caffe2_pb2
from caffe2.python import core, net_drawer, net_printer, visualize, workspace, utils

# Let's run the mobile nets that we generated above so that caffe2 workspace is properly initialized
workspace.RunNetOnce(init_net)
workspace.RunNetOnce(predict_net)

# Caffe2 has a nice net_printer to be able to inspect what the net looks like and identify
# what our input and output blob names are.
print(net_printer.to_string(predict_net))

torch-jit-export_predict = core.Net('torch-jit-export_predict')
torch-jit-export_predict.Transpose(['0'], ['3'], axes=[0, 3, 1, 2])
torch-jit-export_predict.Conv(['3', '1', '2'], ['4'], strides=[1, 1], pads=[0, 0, 0, 0], kernels=[3, 3], group=1, dilations=[1, 1])


In [190]:
# Now, let's also pass in the resized cat image for processing by the model.
workspace.FeedBlob(tf_rep.inputs[0], numpy_input.astype(np.float32))

# run the predict_net to get the model output
workspace.RunNetOnce(predict_net)

# Now let's get the model output blob
caffe2_proto_output = workspace.FetchBlob(tf_rep.outputs[0])

In [191]:
# torch with TF
if not inputs_same(torch_output[0], caffe2_proto_output[0]):
    raise Exception("PyTorch and Caffe2 Proto outputs are not same")

### TF to TFlite

**Check failed: status.ok() Unexpected value for attribute 'data_format'. 
Expected 'NHWC'**

channels first (NCHW) is normally optimal on GPU and channels last (NHWC)
on CPU. The exception is Intel MKL on CPU which is optimal with
channels_last.

At the moment TensorFlow Lite supports only TensorFlow's "NHWC" format, and broadcasting is only support in a limited number of ops (tf.add, tf.mul, tf.sub, and tf.div).

https://www.tensorflow.org/lite/guide/ops_compatibility


When input is "NHWC" there is a different error:

- 2019-03-19 18:24:24.038521: I tensorflow/lite/toco/import_tensorflow.cc:1324] Converting unsupported operation: PyFunc

- 2019-03-19 18:24:24.051107: F tensorflow/lite/toco/import_tensorflow.cc:114] Check failed: attr.value_case() == AttrValue::kType (1 vs. 6)



In [192]:
converter = tf.lite.TFLiteConverter.from_frozen_graph(str(MODELS/f'{arch_name}.pb'),
                                                      tf_rep.tensor_dict[tf_rep.inputs[0]].name.split(":")[:1],
                                                      tf_rep.tensor_dict[tf_rep.outputs[0]].name.split(":")[:1])
tflite_model = converter.convert()
open(MODELS/f"{arch_name}.tflite", "wb").write(tflite_model)

9452